**RUN INSTRUCTIONS**
Prerequisite:
- Clear photo of both player 1 and player 2 hands not crossing the median of the photo.
- Ensure hands are clear and in an appropriate direction such that it is evidently clear from a birds eye view the selection between rock, paper or scissors.

1. Ensure all libraries are installed.
2. Run the following command in the terminal:

        python rps_live_solver.py "image_name_in_directory".jpg

In [ ]:
# Imports
from ultralytics import YOLO
import argparse
import random
# You don't need argparse in the notebook, but keep it if you want to use the main() function structure

# --- 1. CONFIGURATION ---
GESTURES = ["rock", "paper", "scissors"]
GESTURE_ID_MAP = {
    0: 'rock', 
    1: 'paper', 
    2: 'scissors'
}
YOLO_MODEL_PATH = "best.pt"  # Ensure this path is correct

# --- 2. HAND DATA STRUCTURE ---
class Hand:
    def __init__(self, gesture_id, x_center, y_center, width, height):
        self.gesture_id = gesture_id
        self.x_center = x_center
        self.y_center = y_center
        self.width = width
        self.height = height
        self.player = None  
        self.side = None    
    def __repr__(self):
        return (f"Hand(Gesture:{self.gesture_id}, Player:{self.player}, "
                f"Side:{self.side}, Y:{self.y_center:.2f})")

# --- 3. GAME LOGIC FUNCTIONS ---
def beats(a, b):
    # RPS win logic
    return (a == "rock" and b == "scissors") or \
           (a == "scissors" and b == "paper") or \
           (a == "paper" and b == "rock")

def remove_gesture(my_hand, opp_hand):
    # Your complex RPS-Minus-One removal logic with added error checking
    my_set = set(my_hand)
    opp_set = set(opp_hand)

    common = list(my_set & opp_set)
    my_unique = list(my_set - opp_set)
    opp_unique = list(opp_set - my_set)

    if len(common) == 2:
        a, b = list(my_set)
        if beats(a, b):
            return b
        else:
            return a

    if len(common) == 1:
        if len(my_unique) != 1 or len(opp_unique) != 1:
            print("WARNING: Input violates rule (not exactly 2 distinct gestures per player).")
            return random.choice(my_hand)

        common_gesture = common[0]
        my_nc = my_unique[0]
        opp_nc = opp_unique[0]

        if beats(my_nc, opp_nc):
            return my_nc

        if beats(opp_nc, my_nc):
            return common_gesture
    
    # Handle the "No common gesture" case (len(common) == 0)
    if len(common) == 0:
         return random.choice(my_hand) 

    return random.choice(my_hand)

In [ ]:
# --- 4. DATA EXTRACTION AND PARSING ---

def detect_hands(image_path, model_path, gesture_map):
    """Loads model, runs inference, and converts results into Hand objects."""
    try:
        model = YOLO(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        return []

    # Run inference
    results = model(image_path, verbose=False) 
    hands = []
    
    if results and results[0].boxes:
        r = results[0]

        for box in r.boxes:
            x_c_norm, y_c_norm, w_norm, h_norm = box.xywhn[0].tolist() 
            class_id = int(box.cls[0].item())

            if class_id in gesture_map:
                hands.append(Hand(class_id, x_c_norm, y_c_norm, w_norm, h_norm))
    
    return hands

# --- 5. RPS Solver Class ---
class RPSCustomSolver:
    
    def __init__(self, hands):
        self.hands = hands
        self.gesture_names = GESTURE_ID_MAP 

    def _assign_hands_to_players(self):
        opponent_hands = []
        my_hands = []
        
        for hand in self.hands:
            if hand.y_center < 0.5: 
                hand.player = 'OPPONENT'
                opponent_hands.append(hand)
            else:
                hand.player = 'MINE'
                my_hands.append(hand)

        my_hands.sort(key=lambda h: h.x_center)
        if len(my_hands) >= 2:
            my_hands[0].side = 'LEFT'
            my_hands[1].side = 'RIGHT'
        
        opponent_hands.sort(key=lambda h: h.x_center)
        
        self.opponent_hands = opponent_hands
        self.my_hands = my_hands


    def solve(self):
        self._assign_hands_to_players()
        
        if len(self.opponent_hands) < 2 or len(self.my_hands) < 2:
            return f"Error: Model detected {len(self.opponent_hands)} opponent hand(s) and {len(self.my_hands)} of your hand(s). Need two of each for the game."

        my_gesture_names = [self.gesture_names[h.gesture_id] for h in self.my_hands]
        opp_gesture_names = [self.gesture_names[h.gesture_id] for h in self.opponent_hands]
        
        gesture_to_remove = remove_gesture(my_gesture_names, opp_gesture_names)

        hand_to_remove_side = None
        
        # Determine the physical hand to remove (Left or Right)
        # Note: We rely on the sorted my_hands list where index 0 is LEFT and 1 is RIGHT.
        hand_gestures = [self.gesture_names[h.gesture_id] for h in self.my_hands]
        
        if hand_gestures[0] == gesture_to_remove and hand_gestures[1] == gesture_to_remove:
            # Both hands are the same gesture (violates rule, but choose one)
            hand_to_remove_side = 'LEFT'
        elif hand_gestures[0] == gesture_to_remove:
            hand_to_remove_side = 'LEFT'
        elif hand_gestures[1] == gesture_to_remove:
            hand_to_remove_side = 'RIGHT'
        
        if hand_to_remove_side is None:
             return f"Logic Error: Could not find the physical hand matching the required removal gesture: **{gesture_to_remove}**"

        result_message = (
            f"\n--- Game Analysis ---\n"
            f"Your hands: **{my_gesture_names[0].capitalize()}** (Left), **{my_gesture_names[1].capitalize()}** (Right)\n"
            f"Opponent's hands: **{opp_gesture_names[0].capitalize()}**, **{opp_gesture_names[1].capitalize()}**\n"
            f"Custom Logic Determined Remove: **{gesture_to_remove.capitalize()}**\n"
            f"**ACTION: REMOVE your {hand_to_remove_side} hand.**"
        )
        
        return result_message

In [ ]:
# --- 6. MAIN EXECUTION ---

# 1. Define the path to the image you want to test
IMAGE_FILE = 'game_photo_3.jpg' 

# Optional: Display the image to verify the input (requires matplotlib)
# import matplotlib.pyplot as plt
# import cv2
# img = cv2.imread(IMAGE_FILE)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.title(f"Input Image: {IMAGE_FILE}")
# plt.axis('off')
# plt.show()

# 2. Run detection
print(f"Running detection on {IMAGE_FILE} using model {YOLO_MODEL_PATH}...")
hands = detect_hands(IMAGE_FILE, YOLO_MODEL_PATH, GESTURE_ID_MAP)

if not hands:
    print("No hands were detected in the image, or the model failed to load.")
else:
    # 3. Solve the game
    solver = RPSCustomSolver(hands)
    instruction = solver.solve()
    
    print(instruction)

Running detection on game_photo_3.jpg using model best.pt...

--- Game Analysis ---
Your hands: **Paper** (Left), **Rock** (Right)
Opponent's hands: **Paper**, **Paper**
Custom Logic Determined Remove: **Rock**
✅ **ACTION: REMOVE your RIGHT hand.**
